# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686574


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:42,  1.50s/it]

Rendering models:  10%|█         | 3/30 [00:03<00:30,  1.11s/it]

Rendering models:  13%|█▎        | 4/30 [00:03<00:21,  1.18it/s]

Rendering models:  17%|█▋        | 5/30 [00:03<00:16,  1.53it/s]

...

Rendering models:  80%|████████  | 24/30 [00:11<00:01,  5.32it/s]

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  6.75it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  7.10it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  6.29it/s]

Anfrily                               0.000270
zac8151                               0.000357
CThomas                               0.000444
not-logged-in-08025d0dbd12da15cf29    0.000284
not-logged-in-5fb8d79457d3ba01439f    0.000286
not-logged-in-f7e2260ed4a9517a3e4a    0.077078
Dynamatt                              0.000281
Andy_Carleen                          0.000260
not-logged-in-2159616075e1803d6325    0.001185
austingasbarra                        0.000316
Jessica_Korkmaz                       0.001252
c11003498                             0.003172
MilesGray                             0.066398
not-logged-in-5f30b2bb9b7f9d49da41    0.000982
coyotes54                             0.000431
maybeadayoff                          0.000276
pangeli5                              0.000285
acapirala                             0.000287
megamikey0903                         0.000760
Lavadude                              0.036088
ShirleyL                              0.000300
TateNachampas

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())